#### Data Quality

1. Schema válido (Feito)
2. Campos obrigatórios não nulos
3. Chaves únicas
4. Validação de domínio (valores permitidos)
5. Validação de formato (CPF, email, CEP)
6. Regras de negócio
7. Remoção de duplicados
8. Tratamento de inconsistências
9. Criação de métricas de qualidade


**Criando Schemas e Volume Tables para Armazenamento de Dados em DeltaTable**

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS costumers_registrations.silver
COMMENT "Schema com Data Quality Aplicado";

CREATE VOLUME IF NOT EXISTS costumers_registrations.silver.registrations
COMMENT "Volume com Data Quality Aplicado para Cadastros";


**Contrato de Schema - Validação de schema**

In [0]:
# Imports

from pyspark.sql.types import *
from pyspark.testing.utils import assertSchemaEqual
from pyspark.sql.functions import *


In [0]:
# Comparação de schemas esperado x real

schema_expected = StructType([
    StructField('id', StringType(), True),
    StructField('nome', StringType(), True),
    StructField('data_nascimento', DateType(), True),
    StructField('cpf', StringType(),True),
    StructField('cep', StringType(), True),
    StructField('cidade', StringType(), True),
    StructField('estado', StringType(), True),
    StructField('pais', StringType(), True),
    StructField('genero', StringType(), True),
    StructField('telefone', StringType(), True),
    StructField('email', StringType(), True),
    StructField('data_cadastro', DateType(), True)
])

In [0]:
df_registrations = spark.read.parquet('/Volumes/costumers_registrations/raw/registrations')

actual_schema = df_registrations.schema

assertSchemaEqual(actual_schema, schema_expected)

print('TESTE DE SCHEMA')
print(f"Schema Esperado:\n {schema_expected}\n")
print(f"Schema Atual:\n {actual_schema}\n")
print("\nTeste de Schema Concluído\n")

**Campos obrigatórios não nulos**

In [0]:


df_nulls = df_registrations.filter(
    col("id").isNull() |
    col("cpf").isNull() |
    col("email").isNull() |
    col("data_cadastro").isNull() |
    col("data_nascimento").isNull()
)

if df_nulls.count() == 0:
    print("Não há registros inválidos por nulos")
else:
    print(f"\nQuantidade de registros inválidos por nulos: {df_nulls.count()}\n")
